In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_small.xml')

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3756ebf2-eb3c-4e8e-903c-b935b6f99f0a;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 298ms :: artifacts dl 7ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf , explode,lower, rtrim, regexp_replace
from pyspark.sql.types import StringType,ArrayType
import regex

In [4]:
v = df._jdf.schema().treeString()
text_file = open("schema.txt", "w")
n = text_file.write(v)
text_file.close()

In [5]:
from pyspark.sql.functions import col, udf , explode,lower, rtrim,regexp_replace
from pyspark.sql.types import StringType,ArrayType
import regex

# task2 


In [6]:
1+1

2

### The Instructions are unclear the way it is preferred. So if we assume the counts are

In [9]:
def return_links(text):
    innerlink = regex.findall(r'\[\[((?:[^[\]]+|(?R))*+)\]\]', text)
    if innerlink:
        return innerlink
    return None
return_links_udf = udf(lambda text: return_links(text), ArrayType(StringType()))
new_df = df.select(col('title'), col('revision.text._VALUE').alias('text'))
new_df = new_df.withColumn('article', explode(return_links_udf(lower(col('text')))))
new_df = new_df.withColumn('article', regexp_replace(col('article'),".*\#[^\|]+\|?",""))
new_df = new_df.withColumn('article', regexp_replace(col('article'),".*(?!Category)\:[^\|]+\|?",""))
new_df = new_df.withColumn('article', regexp_replace(col('article'),"\|.*",""))
new_df = new_df.filter("article != ''")

In [ ]:
# This is slower process 

In [8]:
new_df.count()

4342885

In [13]:

def return_links(text):
    try:
        results = regex.findall(r'\[\[((?:[^[\]]+|(?R))*+)\]\]', text)
    except:
        return []
    output = []
    for res in results:
        for link in res.split('|'):
            if '#' in link:
                continue
            elif ':' in link and 'Category:' not in link:
                continue
            else:
                output.append(link.lower())
                break
    return output


link_udf = udf(lambda text: return_links(text), ArrayType(StringType()))
new_df = df.withColumn("article", explode(link_udf(col("revision.text._VALUE"))))
new_df = new_df.select(lower(col('title')).alias('title'), 'article').orderBy('title', 'article')

In [14]:
new_df.count()

4413998

In [13]:
new_df.show()

+-------------------+--------------------+
|              title|             article|
+-------------------+--------------------+
|accessiblecomputing|computer accessib...|
|          anarchism|political philosophy|
|          anarchism|      state (polity)|
|          anarchism|   stateless society|
|          anarchism|             anarchy|
|          anarchism|  the globe and mail|
|          anarchism|routledge encyclo...|
|          anarchism|           authority|
|          anarchism|the oxford compan...|
|          anarchism|  family resemblance|
|          anarchism|the oxford compan...|
|          anarchism|oxford university...|
|          anarchism|  mutually exclusive|
|          anarchism|    social anarchism|
|          anarchism|individualist ana...|
|          anarchism| geoffrey ostergaard|
|          anarchism|           left-wing|
|          anarchism|  the new york times|
|          anarchism| anarchist economics|
|          anarchism|       anarchist law|
+----------

In [17]:
new_df.limit(5).toPandas().to_csv('/home/saisur/part2.csv', sep='\t', index=False)

In [17]:
new_df.repartition(10).write.option("delimiter", "\t").csv('part2_small')

In [ ]:
## Python Code final for not splitting
from pyspark.sql.functions import col, udf , explode,lower, rtrim, regexp_replace
from pyspark.sql.types import StringType,ArrayType
import regex

# Get the spark instance and get the file 
spark = SparkSession.builder.getOrCreate()
df = spark.read.format('xml').options(rowTag='page').load('hdfs:/enwiki_test.xml')

# Get the find all  of particular type 
def return_links(text):
    innerlink = regex.findall(r'\[\[((?:[^[\]]+|(?R))*+)\]\]', text)
    if innerlink:
        return innerlink
    return None


return_links_udf = udf(lambda text: return_links(text), ArrayType(StringType()))
new_df = df.select(col('title'), col('revision.text._VALUE').alias('text'))
new_df = new_df.withColumn('article', explode(return_links_udf(lower(col('text')))))
new_df = new_df.withColumn('article', regexp_replace(col('article'),".*\#[^\|]+\|?",""))
new_df = new_df.withColumn('article', regexp_replace(col('article'),".*(?!Category)\:[^\|]+\|?",""))
new_df = new_df.withColumn('article', regexp_replace(col('article'),"\|.*",""))
new_df = new_df.filter("article != ''")
new_df = new_df.select(lower(col('title')).alias('title'), 'article')
new_df.repartition(10).write.option("delimiter", "\t").csv('part2_small')


